<a href="https://colab.research.google.com/github/grifire/Systeme-Solaire/blob/main/1-Recherche%20d'information%20classique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Partie 1. - Recherche d'Information classique

Dans cette partie, nous allons mettre en œuvre des principes et modèles classiques de Recherche d'Information. Le jeu de données est une collection de livres au format texte (.txt) de Henry Rider Haggard. Jetez un œil à ces documents dans le dossier _data_.

En sortie de ce module, vous serez capable de :

- Construire un index inversé ;
- Effectuer des requêtes simples selon le modèle booléen :
- Calculer la pondération des termes selon la méthode TF-IDF ;
- Mettre en œuvre le modèle vectoriel de recherche d'information et y appliquer des requêtes.

### Import des bibliothèques logicielles et configuration

Les lignes suivantes permettent d'instancier un objet de la classe `IRSystem` représentant notre moteur de recherche et de charger les données en RAM.

In [30]:
import os

# Vérifie si le code est exécuté sur Google Colab
if 'COLAB_GPU' in os.environ:
    # Commandes à exécuter uniquement sur Google Colab
    !git clone https://github.com/vincentmartin/tp-information-retrieval-with-llm-student-version.git
    %cd tp-information-retrieval-with-llm-student-version
else:
    # Commandes à exécuter si ce n'est pas sur Google Colab
    print("Pas sur Google Colab, ces commandes ne seront pas exécutées.")

Cloning into 'tp-information-retrieval-with-llm-student-version'...
remote: Enumerating objects: 2254, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 2254 (delta 24), reused 33 (delta 17), pack-reused 2212 (from 1)
Receiving objects: 100% (2254/2254), 13.92 MiB | 13.27 MiB/s, done.
Resolving deltas: 100% (26/26), done.
/content/tp-information-retrieval-with-llm-student-version/tp-information-retrieval-with-llm-student-version


#### Chargement des données

Les lignes ci-dessous permettent de charger les données qui sont un ensemble de 60 livres au format texte (.txt) d'[Henry Rider Haggard](https://fr.wikipedia.org/wiki/Henry_Rider_Haggard).


In [4]:
from classic_ir.IRSystem import *

# !rm -rf ./data/RiderHaggard/stemmed
ir_system = IRSystem()
ir_system.read_data('./data/RiderHaggard') # chargement des données et prétraitement des documents (stemming).

/content/tp-information-retrieval-with-llm-student-version/classic_ir/IRSystem.py:35: SyntaxWarning: invalid escape sequence '\d'
  title_pattern = re.compile('(.*) \d+\.txt')


Reading in documents...
Stemming Documents...
Allan Quatermain 711.txt
    Doc 1 of 60: Allan Quatermain
Jess 5898.txt
    Doc 2 of 60: Jess
Ayesha, the Return of She 5228.txt
    Doc 3 of 60: Ayesha, the Return of She
Allan and the Ice Gods (1927) 0200201.txt
    Doc 4 of 60: Allan and the Ice Gods (1927)
Lysbeth, a Tale of the Dutch 5754.txt
    Doc 5 of 60: Lysbeth, a Tale of the Dutch
The People of the Mist 6769.txt
    Doc 6 of 60: The People of the Mist
Queen of the Dawn (1925) 0200381.txt
    Doc 7 of 60: Queen of the Dawn (1925)
Long Odds 1918.txt
    Doc 8 of 60: Long Odds
The Mahatma and the Hare 2764.txt
    Doc 9 of 60: The Mahatma and the Hare
Dawn 10892.txt
    Doc 10 of 60: Dawn
Marie An Episode in The Life of the late Allan Quatermain 1690.txt
    Doc 11 of 60: Marie An Episode in The Life of the late Allan Quatermain
Child of Storm 1711.txt
    Doc 12 of 60: Child of Storm
Stella Fregelius 6051.txt
    Doc 13 of 60: Stella Fregelius
Red Eve 3094.txt
    Doc 14 of 60: R

### Exercice 1. - Construction de l'index inversé

Ce premier exercice a pour objectif de construire l'index inversé non positionnel. L'attribut `self.inverted_index` est un tableau associatif qui associe une liste d'entiers (docIDs) à un mot (word).

Documentation ici https://docs.python.org/3/library/collections.html#collections.defaultdict.

Exercice : modifier la fonction `index` pour calculer l'index inversé.

Le résultat ci-dessous indique que vous avez réussi.
```
===== Running tests =====
Inverted Index Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [32]:
# Exercice 1. Indexation

def index(self):
    """
    Construit l'index inversé et le stocke dans self.inverted_index.

    Dans le code ci-dessous, seul le dictionnaire des tokens est construit. Les postings lists sont vides.
    """
    print("Indexing...")
    self.tf = defaultdict(Counter) # tf est un dictionnaire qui à un document associe un Counter de mots.
    inverted_index = defaultdict(list) # inverted_index est un dictionnaire qui à un mot associe une liste de documents.
    for word in self.vocab:
        inverted_index[word] = list()
    #print(self.docs[0])
    i = 0
    for doc in self.docs:
      for word in doc:
        if i not in inverted_index[word]:
          inverted_index[word].append(i)
      i+=1



    self.inverted_index = inverted_index

# Ne pas modifier les lignes ci-dessous
IRSystem.index = index
ir_system.index()
run_tests(ir_system, part=0)

Indexing...
===== Running tests =====
Inverted Index Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000


### Exercice 2. - Recherche booléenne

Ce deuxième exercice a pour objectif d'implémenter la recherche booléenne. La requête `query` est une liste de mots _lemmatisés_ et l'algorithme doit rechercher les documents qui contiennent TOUS ces mots.


Exercice : modifier la fonction `boolean_retrieve` pour implémenter la recherche booléenne.


Le résultat ci-dessous indique que vous avez réussi.
```
===== Running tests =====
Boolean Retrieval Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [34]:
# Exercice 2. Recherche booléenne
def boolean_retrieve(self, query):
    """
    A partir d'une requête sous la forme d'une liste de mots *lemmatisés*,
    retourne la liste des documents dans lesquels *tous* ces mots apparaissent (ie une requête AND).
    Retourne une liste vide si la requête ne retourne aucun document.

    Dans le code ci-dessous, tous les documents répondent.
    """
    docs = list()
    i = 0
    for doc in self.docs:
        docs.append(i)
        i+=1
    for word in query :
      i=0
      for doc in self.docs :
        if word not in doc and i in docs:
          docs.remove(i)
        i+=1
    return docs

# Ne pas modifier les lignes ci-dessous
IRSystem.boolean_retrieve = boolean_retrieve
run_tests(ir_system, part=1)

===== Running tests =====
Boolean Retrieval Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000


### Exercice 3. - Calcul des poids TF-IDF des termes dans les documents

Dans ce troisième exercice, l'objectif est de pré-calculer les poids TF-IDF pour chaque terme dans chaque document. Pour ce faire, appliquer la formule vue en cours. Utiliser le logarithme en base 10.


Exercice : modifier la fonction `boolean_retrieve` pour implémenter la recherche booléenne.

Le résultat ci-dessous indique que vous avez réussi.
```
Calculating tf-idf...
===== Running tests =====
TF-IDF Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [64]:
from collections import Counter
from math import *
# Exercice 3. calcul des scores tf-idf
def compute_tfidf(self):
    """
    Calcule les scores tf-idf pour tous les mots de tous les documents et les stocke dans self.tfidf.

    Dans le code ci-dessous, les scores tf-idf sont tous nuls.
    """
    print("Calculating tf-idf...")
    print(Counter(self.docs[0]))
    self.tfidf = defaultdict(Counter)
    N = len(self.docs)  # N = nombre de documents
    print(len(self.vocab))
    n = 0
    idf = []
    tmp_tf = []
    for word in self.vocab :
      idf.append(log10(N/len(self.inverted_index[word])))
    for i in range(N):
      tmp_tf.append(Counter(self.docs[i]))
    print("end = ", tmp_tf[0]["a"])
    for word in self.vocab:
        for i in range(N):

            try:
                self.tfidf[i][word] = (1+log10(tmp_tf[i][word])) * idf[n]
            except ValueError:
                self.tfidf[i][word] = 0.
            #print("n = ",n," i = ",i, " tfidf = ", self.tfidf[i][word])
        n+=1

# Ne pas modifier les lignes ci-dessous
IRSystem.compute_tfidf = compute_tfidf
ir_system.compute_tfidf()
run_tests(ir_system, part=2)

Calculating tf-idf...
Counter({'the': 9033, 'of': 5209, 'and': 3305, 'to': 3234, 'a': 2640, 'in': 2266, 'it': 1812, 'that': 1715, 'is': 1408, 'i': 1343, 'on': 1066, 'wa': 1065, 'with': 942, 'which': 913, 'as': 908, 'thi': 833, 'for': 830, 'by': 827, 'be': 795, 'at': 772, 'have': 658, 'we': 658, 'he': 632, 'or': 607, 'not': 607, 'their': 570, 'hi': 558, 'thei': 556, 'from': 518, 'were': 508, 'them': 498, 'but': 492, 'ar': 481, 'who': 430, 'so': 416, 'upon': 409, 'had': 387, 'there': 385, 'been': 337, 'an': 328, 'place': 326, 'these': 325, 'our': 325, 'all': 315, 'some': 308, 'my': 303, 'us': 286, 'veri': 282, 'here': 268, 'when': 262, 'would': 258, 'more': 258, 'other': 257, 'ha': 255, 'where': 254, 'no': 244, 'about': 242, 'mani': 222, 'me': 220, 'out': 216, 'up': 216, 'now': 212, 'great': 210, 'dai': 204, 'time': 203, 'what': 197, 'also': 197, 'such': 197, 'year': 196, 'if': 196, 'those': 194, 'into': 193, 'him': 193, 'must': 192, 'mai': 189, 'even': 184, 'most': 177, 'her': 177, 'cou

### Exercice 4. - Calcul de la similarité cosinus.

Dans ce troisième exercice, l'objectif est de calculer la similarité cosinus entre le vecteur requête `query` et chaque vecteur document. Pour ce faire, appliquer la formule vue en cours en considérant les éléments suivants :
- Ne considérer que la mesure TF pour calculer le poids des termes de la requête (on n'utilise pas IDF).
- Utiliser le logarithme en base 10.

Exercice : modifier la fonction `rank_retrieve` pour implémenter la recherche booléenne.

Le résultat ci-dessous indique que vous avez réussi.
```
===== Running tests =====
Cosine Similarity Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [99]:
# Exercice 4. Similarité cosinus
def rank_retrieve(self, query):
    """
    A partir d'une requête (une liste de mots), retourne une liste de documents (classés par docID) et de scores pour la requête en appliquant la simalirité cosinus.

    Dans l'exemple ci-dessous. C'est la mesure de Jaccard qui est utilisée.
    """
    scores = [0.0 for _ in range(len(self.docs))]

    query_set = set(query)
    union = 0
    #for d in self.docs:

    for d in range(len(self.docs)):
        # doc_set = set(self.docs[d])
        # intersection = len(query_set & doc_set)
        #union = len(query_set | doc_set)
        # Calcul de la similarité Jaccard
        union = 0
        for word in set(self.docs[d]) :
            union += self.tfidf[d][word]**2
        union = union ** 0.5
        intersection = 0
        for word in query :
          intersection += self.tfidf[d][word]
        scores[d] = intersection / union if union != 0 else 0.0

    # Tri des scores
    ranking = [idx for idx, sim in sorted(enumerate(scores),
                                        key=lambda pair: pair[1], reverse=True)]
    results = []
    for i in range(10):
        results.append((ranking[i], scores[ranking[i]]))
    print("mon résultat : ",results)
    return results

# Ne pas modifier les lignes ci-dessous
IRSystem.rank_retrieve = rank_retrieve
run_tests(ir_system, part=3)

===== Running tests =====
Cosine Similarity Test
mon résultat :  [(56, 0.010676611271744125), (25, 0.008463116269726578), (49, 0.008462987843582521), (37, 0.008395866580849027), (18, 0.008360056776471418), (54, 0.008320124278304232), (5, 0.00831602065054907), (41, 0.008312726995818033), (29, 0.007852092147131635), (44, 0.007814399700034932)]
mon résultat :  [(59, 0.05041279333939146), (31, 0.042933912797405153), (16, 0.042457225372370705), (36, 0.0391023868431655), (3, 0.032503697016589644), (45, 0.03200276000633722), (40, 0.02835346095334227), (1, 0.022988250326529514), (6, 0.022750210842425178), (48, 0.022439179053556458)]
mon résultat :  [(15, 0.034057894475520906), (41, 0.02218901367816246), (39, 0.02180077067566579), (13, 0.016669193743447722), (16, 0.010152619001197604), (52, 0.00935284283446791), (27, 0.00861473088344379), (50, 0.008208957726149083), (29, 0.00803544943397247), (5, 0.00746212911745868)]
mon résultat :  [(24, 0.015298081869538089), (21, 0.014226522006871013), (45,